## Recap: 

By using a list of 83 sectors and 204 industries we were able to get an accuracy of 50% and 44%. This was achieved after sepeerating the API call into two. 

## Next steps 

Our next step would be to try and condense these lists, which we hope will increase accuracy and decrese waiting time. 

### Condensing the lists

Our challange advisors gave us a list of industries that their vendor uses, we will try to use the API with those industries as iptions and see how the accuracy changes. 

We will do this by: 

1. Creating a map from all existing industries to the ones we want to use. 
2. Applying the map to the data
3. Feed the new list and companies to the LLM 
4. Comapre the results and see accuracy with this new list

## Preparing the data set

We filter by only having full time and US jobs, then sepesrate the JSON company profile into sector and indsutry and drop company profile and job ID features.

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [35]:
df = pd.read_csv("job_descriptions.csv")

In [36]:
feature_to_filter_type = "Work Type"
values_to_keep_type = ["Full-Time"]

df_filtered_type = df[df[feature_to_filter_type].isin(values_to_keep_type)]

In [37]:
feature_to_filter_country = "Country"
values_to_keep_country = ["USA"]

df_filtered_country = df[df[feature_to_filter_country].isin(values_to_keep_country)]

In [38]:
import json

def extract_sector_industry(row):
  try:
    profile = json.loads(row['Company Profile'])
    return profile.get('Sector'), profile.get('Industry')
  except:
    return None, None

df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')

/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_50571/3398901514.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')
/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_50571/3398901514.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')


In [39]:
df_filtered_country.drop(columns=['Company Profile'], inplace=True)
df_filtered_country.drop(columns=['Job Id']).describe()

/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_50571/2787395556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country.drop(columns=['Company Profile'], inplace=True)


,latitude,longitude,Company Size
count,7.431000e+03,7.431000e+03,7431.000000
mean,3.709020e+01,-9.571290e+01,73073.289194
std,2.934739e-12,7.446989e-12,35201.315418
min,3.709020e+01,-9.571290e+01,12650.000000
25%,3.709020e+01,-9.571290e+01,42393.500000
50%,3.709020e+01,-9.571290e+01,72502.000000
75%,3.709020e+01,-9.571290e+01,103686.500000
max,3.709020e+01,-9.571290e+01,134824.000000


In [40]:
df_filtered_country.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Sector,Industry
156,579575048679091,3 to 9 Years,MCA,$59K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,126334,...,Network Engineer,Wireless Network Engineer,USAJOBS,"Wireless Network Engineers design, implement, ...","{'Flexible Spending Accounts (FSAs), Relocatio...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Lendlease Group,Real Estate/Construction,Real Estate
270,288941837754490,4 to 15 Years,B.Tech,$57K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,35869,...,Landscape Architect,Urban Planner,The Muse,Develop and implement urban planning strategie...,"{'Employee Referral Programs, Financial Counse...",Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",United Technologies Corporation,Aerospace/Building Systems,Aerospace and Defense
1140,2802376880223274,0 to 9 Years,B.Tech,$63K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,80163,...,UX Researcher,UX Strategist,SimplyHired,"Develop user experience (UX) strategies, creat...","{'Transportation Benefits, Professional Develo...",User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,International Flavors & Fragrances,Flavors & Fragrances,Chemicals
1591,2393222411935908,5 to 11 Years,MCA,$55K-$124K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,79017,...,HR Coordinator,Benefits Coordinator,Internships.com,Benefits Coordinators administer employee bene...,"{'Transportation Benefits, Professional Develo...",Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",KeyCorp,Financial Services,Commercial Banks
1609,2402523179167443,0 to 10 Years,MCA,$65K-$91K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,65144,...,Landscape Architect,Environmental Designer,USAJOBS,"Design outdoor spaces, parks, and landscapes w...","{'Childcare Assistance, Paid Time Off (PTO), R...",Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,Exelon,Energy,Utilities: Gas and Electric


In [41]:
df_filtered_country.shape

(7431, 24)

In [42]:
df_filtered_country['Industry'].isnull().sum()

40

In [43]:
df_filtered_country['Sector'].isnull().sum()

40

# Dealing with nulls

We decide to remove null values in this case since there is only 40 of them

In [44]:
df_filtered_country = df_filtered_country.dropna(subset=['Sector', 'Industry'])

In [45]:
df.to_csv("filtered_country_work_type.csv", index=False)

# Mapping

Load the real industy sectors and map them to the current sectors. Then add a new collumn with these mapped sectors

In [46]:
real_salary_sectors = [
    "Aerospace",
    "Construction",
    "Consumer Goods",
    "Coorporate Services",
    "Defense",
    "Design",
    "Education",
    "Energy and Mining",
    "Engineering",
    "Entertainment",
    "Finance",
    "Hardware and Newtowrking",
    "Healthcare",
    "Manufacturing",
    "Media and Communications",
    "Nonprofit",
    "Public Administration",
    "Real Estate",
    "Recreation and Travel",
    "Retail",
    "Software and IT Services",
    "Telecommunications",
    "Transportation and Logistics",
    "Wellness and Fitness"
]

In [47]:
data_sectors = df_filtered_country["Sector"].unique()
data_sectors.sort()
print(data_sectors)

['Advertising & Marketing' 'Advertising and Marketing'
 'Aerospace & Defense' 'Aerospace and Defense'
 'Aerospace/Building Systems' 'Aerospace/Defense' 'Agriculture'
 'Agrochemicals' 'Airlines' 'Aluminum' 'Apparel' 'Apparel and Footwear'
 'Appliances' 'Appliances and Electronics' 'Asset Management' 'Automotive'
 'Automotive Parts' 'Automotive/Motorcycles' 'Aviation and Travel'
 'Banking' 'Banking/Financial Services' 'Beauty' 'Beauty/Cosmetics'
 'Beverage' 'Beverage/Alcohol' 'Beverages' 'Biotechnology'
 'Building Materials' 'Building Materials & Equipment' 'Business Services'
 'Cement & Aggregates' 'Chemicals' 'Chemicals and Energy'
 'Communication Equipment' 'Communication Services' 'Conglomerate'
 'Construction' 'Construction & Engineering'
 'Construction and Engineering' 'Construction/Building Materials'
 'Construction/Infrastructure' 'Consumer Discretionary' 'Consumer Goods'
 'Consumer Goods/Homecare' 'Data and Analytics' 'Diversified' 'E-commerce'
 'E-commerce/Online Marketplace' '

In [48]:
print(data_sectors.shape)

(204,)


Since there are so many different possible value we deicide to use an OLLAMA call to map each unique sector to a real salary sector 

In [49]:
import requests
url = "http://localhost:11434/api/chat"

In [50]:
def mapSectors(prompt):
    real_salary_sectors_str = ", ".join(real_salary_sectors)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": f"Take {prompt} and map it to the closest sector from the following list: {real_salary_sectors_str}. Respond with only the sector that you think is the closest match. Do not add anything else to you response."

            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    return response


In [51]:
print(mapSectors("Advertising & Marketing"))

Media and Communications


In [52]:
mapped_sectors = []

for sector in data_sectors:
    new_sector = mapSectors(sector)
    mapped_sectors.append(new_sector)
    print(f"{sector} -> {new_sector}")

Advertising & Marketing -> Media and Communications
Advertising and Marketing -> Media and Communications
Aerospace & Defense -> Engineering
Aerospace and Defense -> Manufacturing
Aerospace/Building Systems -> Construction
Aerospace/Defense -> Defense
Agriculture -> Engineering
Agrochemicals -> Manufacturing
Airlines -> Aerospace
Aluminum -> Aerospace
Apparel -> Consumer Goods
Apparel and Footwear -> Consumer Goods
Appliances -> Consumer Goods
Appliances and Electronics -> Consumer Goods
Asset Management -> Finance
Automotive -> Transportation and Logistics
Automotive Parts -> Manufacturing
Automotive/Motorcycles -> Transportation and Logistics
Aviation and Travel -> Transportation and Logistics
Banking -> Finance
Banking/Financial Services -> Finance
Beauty -> Entertainment
Beauty/Cosmetics -> Consumer Goods
Beverage -> Consumer Goods
Beverage/Alcohol -> Consumer Goods
Beverages -> Beverages -> Consumer Goods
Biotechnology -> Healthcare
Building Materials -> Construction
Building Mate

In [53]:
print(mapped_sectors)

['Media and Communications', 'Media and Communications', 'Engineering', 'Manufacturing', 'Construction', 'Defense', 'Engineering', 'Manufacturing', 'Aerospace', 'Aerospace', 'Consumer Goods', 'Consumer Goods', 'Consumer Goods', 'Consumer Goods', 'Finance', 'Transportation and Logistics', 'Manufacturing', 'Transportation and Logistics', 'Transportation and Logistics', 'Finance', 'Finance', 'Entertainment', 'Consumer Goods', 'Consumer Goods', 'Consumer Goods', 'Beverages -> Consumer Goods', 'Healthcare', 'Construction', 'Manufacturing', 'Software and IT Services', 'Manufacturing', 'Manufacturing', 'Energy and Mining', 'Media and Communications', 'Media and Communications', 'Conglomerate: Manufacturing', 'Construction', 'Engineering', 'Engineering', 'Construction', 'Construction/Infrastructure -> Construction', 'Consumer Discretionary → Entertainment', 'Consumer Goods -> Consumer Goods', 'Consumer Goods/Homecare → Consumer Goods', 'Data and Analytics: Software and IT Services', 'Finance',

In [54]:
# Map original sectors to mapped sectors
sector_mapping = dict(zip(data_sectors, mapped_sectors))
# Print so its easier to read 
for key, value in sector_mapping.items():
    # print only the ones where the value is not in real_salary_sectors
    if value not in real_salary_sectors:
        # Put these sectors through the API again 
        new_sector = mapSectors(key)
        value = new_sector
        print(f"{key} -> {value}")

Beverages -> Consumer Goods
Conglomerate -> Manufacturing
Construction/Infrastructure -> Construction/Infrastructure -> Construction
Consumer Discretionary -> Entertainment
Consumer Goods -> Consumer Goods -> Consumer Goods
Consumer Goods/Homecare -> Consumer Goods/Homecare -> Consumer Goods
Data and Analytics -> Software and IT Services
Energy -> Energy
Energy/Infrastructure -> Energy/Infrastructure -> Energy and Mining
Energy/Oil and Gas -> Energy/Oil and Gas -> Energy and Mining
Engineering -> Engineering: Manufacturing
Engineering & Construction -> Engineering & Construction -> Construction
Engineering Services -> Engineering Services
Entertainment/Media Streaming -> Media and Communications
Fertilizers -> Agriculture
Financials -> Finance
Food -> Consumer Goods
Food and Beverage -> Food and Beverage → Consumer Goods
Health Care -> Health Care: Healthcare
Healthcare -> Software and IT Services
Healthcare Services -> Healthcare Services: Healthcare
Healthcare Technology -> Finance
H

In [55]:
sum = 0
for key,value in sector_mapping.items():
    if value not in real_salary_sectors:
        print(f"{key} -> {value}")
        sum += 1

Beverages -> Beverages -> Consumer Goods
Conglomerate -> Conglomerate: Manufacturing
Construction/Infrastructure -> Construction/Infrastructure -> Construction
Consumer Discretionary -> Consumer Discretionary → Entertainment
Consumer Goods -> Consumer Goods -> Consumer Goods
Consumer Goods/Homecare -> Consumer Goods/Homecare → Consumer Goods
Data and Analytics -> Data and Analytics: Software and IT Services
Energy -> Energy
Energy/Infrastructure -> Energy/Infrastructure -> Energy and Mining
Energy/Oil and Gas -> Energy/Oil and Gas -> Energy and Mining
Engineering -> Engineering: Manufacturing
Engineering & Construction -> Engineering & Construction → Construction
Engineering Services -> Engineering Services: Engineering
Entertainment/Media Streaming -> Media
Fertilizers -> Agriculture
Financials -> Financials: Finance
Food -> Food -> Consumer Goods
Food and Beverage -> Food and Beverage: Consumer Goods
Health Care -> Healthcare -> Healthcare
Healthcare -> Healthcare -> Healthcare
Healt

In [56]:
print(sum)

48


# First Results

After putting the sectors through the API 50 of them wwre mapped to something that were not in the list, these can be fixed manually

In [127]:
# Manually map the remaining sectors that were not mapped by the API
# For a key manually change the value
sector_mapping["Aerospace/Building Systems"] = "Aerospace"
sector_mapping["Beverages"] = "Consumer Goods"
sector_mapping["Consumer Discretionary"] = "Consumer Goods"
sector_mapping["Consumer Goods/Homecare"] = "Consumer Goods"
sector_mapping["Electronics"] = "Hardware and Newtowrking"
sector_mapping["Engineering"] = "Engineering"
sector_mapping["Engineering & Construction"] = "Engineering"
sector_mapping["Engineering Services"] = "Engineering"
sector_mapping["Financials"] = "Finance"
sector_mapping["Food"] = "Consumer Goods"
sector_mapping["Food and Agriculture"] = "Consumer Goods"
sector_mapping["Food and Beverage/Spices"] = "Consumer Goods"
sector_mapping["Health Care"] = "Healthcare"
sector_mapping["Healthcare Services"] = "Healthcare"
sector_mapping["Healthcare Technology"] = "Healthcare"
sector_mapping["Healthcare/Hospitals"] = "Healthcare"
sector_mapping["Healthcare/Technology"] = "Healthcare"
sector_mapping["Lab Equipment"] = "Healthcare"
sector_mapping["Lighting and Technology"] = "Hardware and Newtowrking"
sector_mapping["Materials"] = "Manufacturing"
sector_mapping["Media"] = "Media and Communications"
sector_mapping["Media & Entertainment"] = "Media and Communications"
sector_mapping["Media and Entertainment"] = "Media and Communications"
sector_mapping["Professional Services"] = "Coorporate Services"
sector_mapping["Real Estate"] = "Real Estate"
sector_mapping["Real Estate/Construction"] = "Real Estate"
sector_mapping["Real Estate/Property Development"] = "Real Estate"
sector_mapping["Retail and Conglomerate"] = "Retail"
sector_mapping["Conglomerate"] = "Retail"
sector_mapping["Semiconductor"] = "Hardware and Newtowrking"
sector_mapping["Software"] = "Software and IT Services"
sector_mapping["Software and Technology"] = "Software and IT Services"
sector_mapping["Staffing"] = "Coorporate Services"
sector_mapping["Technology and Electronics"] = "Hardware and Newtowrking"
sector_mapping["Technology and Entertainment"] = "Media and Communications"
sector_mapping["Technology and Software"] = "Software and IT Services"
sector_mapping["Technology and Telecommunications"] = "Telecommunications"
sector_mapping["Technology/Software"] = "Software and IT Services"
sector_mapping["Transportation and Rail"] = "Transportation and Logistics"
sector_mapping["Transportation/Airlines"] = "Transportation and Logistics"
sector_mapping["Transportation/Airports"] = "Transportation and Logistics"
sector_mapping["Transportation/Logistics"] = "Transportation and Logistics"
sector_mapping["Transportation/Railways"] = "Transportation and Logistics"
sector_mapping["Transportation/Ridesharing"] = "Transportation and Logistics"
sector_mapping["Travel/Cruise Lines"] = "Recreation and Travel"
sector_mapping["Restaurants"] = "Recreation and Travel"
sector_mapping["Services"] = "Coorporate Services"
sector_mapping["Utilities"] = "Energy and Mining"


sector_mapping["Construction/Infrastructure"] = "Construction"
sector_mapping["Consumer Goods"] = "Consumer Goods"
sector_mapping["Data and Analytics"] = "Software and IT Services"
sector_mapping["Energy"] = "Energy and Mining"
sector_mapping["Energy/Infrastructure"] = "Energy and Mining"
sector_mapping["Energy/Oil and Gas"] = "Energy and Mining"
sector_mapping["Entertainment/Media Streaming"] = "Media and Communications"
sector_mapping["Fertilizers"] = "Manufacturing"
sector_mapping["Food and Beverage"] = "Consumer Goods"
sector_mapping["Healthcare"] = "Healthcare"
sector_mapping["Manufacturing/Building Systems"] = "Manufacturing"
sector_mapping["Real Estate/Logistics"] = "Real Estate"
sector_mapping["Retail/Home Improvement"] = "Retail"
sector_mapping["Specialty Chemicals"] = "Manufacturing"
sector_mapping["Transportation/Infrastructure"] = "Transportation and Logistics"
sector_mapping["Travel and Booking"] = "Recreation and Travel"

sum = 0
for key, value in sector_mapping.items():
    if value not in real_salary_sectors:
        print(f"{key} -> {value}")
        sum += 1

print(sum)


0


# Applying the mapping 

Now every unique sector is applied to a real salary sector. With this we will create a new collumn next to the original sector with the new sector

In [128]:
# Map the sectors in the dataframe by adding it to a new column
df_filtered_country["Real-Salary-Industry"] = df_filtered_country["Sector"].map(sector_mapping)

In [129]:
df_filtered_country.head(10)

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Sector,Industry,Real-Salary-Industry
156,579575048679091,3 to 9 Years,MCA,$59K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,126334,...,Wireless Network Engineer,USAJOBS,"Wireless Network Engineers design, implement, ...","{'Flexible Spending Accounts (FSAs), Relocatio...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Lendlease Group,Real Estate/Construction,Real Estate,Real Estate
270,288941837754490,4 to 15 Years,B.Tech,$57K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,35869,...,Urban Planner,The Muse,Develop and implement urban planning strategie...,"{'Employee Referral Programs, Financial Counse...",Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",United Technologies Corporation,Aerospace/Building Systems,Aerospace and Defense,Aerospace
1140,2802376880223274,0 to 9 Years,B.Tech,$63K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,80163,...,UX Strategist,SimplyHired,"Develop user experience (UX) strategies, creat...","{'Transportation Benefits, Professional Develo...",User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,International Flavors & Fragrances,Flavors & Fragrances,Chemicals,Consumer Goods
1591,2393222411935908,5 to 11 Years,MCA,$55K-$124K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,79017,...,Benefits Coordinator,Internships.com,Benefits Coordinators administer employee bene...,"{'Transportation Benefits, Professional Develo...",Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",KeyCorp,Financial Services,Commercial Banks,Finance
1609,2402523179167443,0 to 10 Years,MCA,$65K-$91K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,65144,...,Environmental Designer,USAJOBS,"Design outdoor spaces, parks, and landscapes w...","{'Childcare Assistance, Paid Time Off (PTO), R...",Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,Exelon,Energy,Utilities: Gas and Electric,Energy and Mining
1668,2429224065703046,4 to 10 Years,B.Com,$56K-$87K,"Washington, D.C.",USA,37.0902,-95.7129,Part-Time,49989,...,User Experience Designer,Jobs2Careers,User Experience Designers create intuitive and...,"{'Health Insurance, Retirement Plans, Paid Tim...",User-centered design principles UX/UI design t...,Design and improve user experiences (UX) by cr...,Zoetis,Healthcare,Pharmaceuticals,Healthcare
1860,1167270963119935,0 to 8 Years,M.Com,$55K-$110K,"Washington, D.C.",USA,37.0902,-95.7129,Contract,131825,...,Front-End Developer,Stack Overflow Jobs,A Front-End Developer specializes in creating ...,"{'Employee Referral Programs, Financial Counse...","Front-end web development HTML, CSS, JavaScrip...",Develop and maintain user interfaces for websi...,Southwest Airlines,Airlines,Airlines,Aerospace
1923,1269600252034514,5 to 13 Years,B.Com,$56K-$99K,"Washington, D.C.",USA,37.0902,-95.7129,Full-Time,127065,...,Wedding Planner,SimplyHired,Wedding Planners specialize in organizing wedd...,"{'Employee Assistance Programs (EAP), Tuition ...",Wedding planning Venue selection Catering and ...,"Specialize in wedding planning, assisting coup...",Diageo,Beverages,Beverages,Consumer Goods
2236,665688551170300,1 to 11 Years,PhD,$62K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Part-Time,21532,...,Residential Interior Designer,The Muse,A Residential Interior Designer plans and desi...,"{'Legal Assistance, Bonuses and Incentive Prog...",Residential interior design Space planning Col...,Plan and design interiors for residential spac...,PBF Energy,Energy,Petroleum Refining,Energy and Mining
2278,1705438618009444,2 to 13 Years,B.Tech,$55K-$126K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,40540,...,Fine Arts Ins

# Test for accuracy 

We will preform the same original guess with the API just changing the list that the API can access to be the real salary industry, then test the accuracy.

In [60]:
def getIndustry(prompt):
    real_salary_industry_str = ", ".join(real_salary_sectors)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                # What we are asking the API, will be given the name of company in the prompt
                "content": f"Identify the industry for the company named {prompt}. " 
                            "Respond strictly in the format: 'Industry: [best matching industry]'. "
                            f"Select the most appropriate industry from this list: {real_salary_industry_str}." 
                            "Do not include any additional information or commentary."
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    # extract sector and industry
    try:
        industry = str(response).split("Industry: ")[1].split("]")[0].strip()
    except IndexError:
        industry = "Unknown",

    return industry

In [61]:
reponse_industry = getIndustry("Apple")
print(reponse_industry)

Consumer Goods


In [130]:
df_filtered_country.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Sector,Industry,Real-Salary-Industry
156,579575048679091,3 to 9 Years,MCA,$59K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,126334,...,Wireless Network Engineer,USAJOBS,"Wireless Network Engineers design, implement, ...","{'Flexible Spending Accounts (FSAs), Relocatio...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Lendlease Group,Real Estate/Construction,Real Estate,Real Estate
270,288941837754490,4 to 15 Years,B.Tech,$57K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,35869,...,Urban Planner,The Muse,Develop and implement urban planning strategie...,"{'Employee Referral Programs, Financial Counse...",Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",United Technologies Corporation,Aerospace/Building Systems,Aerospace and Defense,Aerospace
1140,2802376880223274,0 to 9 Years,B.Tech,$63K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,80163,...,UX Strategist,SimplyHired,"Develop user experience (UX) strategies, creat...","{'Transportation Benefits, Professional Develo...",User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,International Flavors & Fragrances,Flavors & Fragrances,Chemicals,Consumer Goods
1591,2393222411935908,5 to 11 Years,MCA,$55K-$124K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,79017,...,Benefits Coordinator,Internships.com,Benefits Coordinators administer employee bene...,"{'Transportation Benefits, Professional Develo...",Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",KeyCorp,Financial Services,Commercial Banks,Finance
1609,2402523179167443,0 to 10 Years,MCA,$65K-$91K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,65144,...,Environmental Designer,USAJOBS,"Design outdoor spaces, parks, and landscapes w...","{'Childcare Assistance, Paid Time Off (PTO), R...",Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,Exelon,Energy,Utilities: Gas and Electric,Energy and Mining


In [134]:
# Save this new data set to a new csv file
df_filtered_country.to_csv("mapped_sectors.csv", index=False)

In [ ]:
test_sizes = [300]
times = []
accuracies = []

In [151]:
import random
import time

for test_size in test_sizes:
    print(f"Testing with {test_size} companies")
    print("=====================================")
    start_time = time.time()
    # Pick 200 random companies
    random_companies = random.sample(list(df_filtered_country["Company"]), test_size)
    # Get the industry for each company
    industries = []
    companies = {}
    actual_industries = []
    print("Predicting industries for companies...")
    for company in random_companies:
        # Make the prediction
        predicted_industry = getIndustry(company)
        print(f"{company} -> {predicted_industry}")
        # Append the prediction to the industry list
        industries.append(predicted_industry)
        # Add the company and the prediction to the dictionary
        companies[company] = predicted_industry
        # Get the actual industry
        actual_industry = df_filtered_country['Real-Salary-Industry'][df_filtered_country['Company'] == company].values[0] # Get the actual industry
        # Append the actual industry to the list
        actual_industries.append(actual_industry)
    end_time = time.time()
    times.append(end_time - start_time)
    print("=====================================")
    print(f"Number of companies: {test_size}")
    print(f"Time taken: {end_time - start_time} seconds")
    # Check accuracy 
    correct = 0
    for i in range(test_size):
        if industries[i] == actual_industries[i]:
            correct += 1
    accuracy = correct / test_size
    accuracies.append(accuracy)
    print(f"Accuracy for {test_size} companies: {accuracy}")

Testing with 200 companies
Predicting industries for companies...
Fresenius SE & Co. KGaA -> Healthcare
Nationwide -> Insurance
Deere -> Manufacturing
Dollar General -> Retail
BASF SE -> Chemicals
Sinopec -> Energy and Mining
Coty Inc. -> Consumer Goods
Albemarle -> Manufacturing
GXO Logistics -> Transportation and Logistics
HDFC Asset Management Company -> Finance
Cairn Energy -> Energy and Mining
Mondelez International -> Consumer Goods
Qube Holdings -> Engineering
Informa -> Media and Communications
Southwest Airlines -> Transportation and Logistics
Domino's Pizza Group -> Retail
Tractor Supply -> Retail
Regeneron Pharmaceuticals -> Healthcare
Kinder Morgan -> Energy and Mining
The Walt Disney Company -> Entertainment
HSBC (Hongkong and Shanghai Banking Corporation) -> Finance
Australia and New Zealand Banking Group (ANZ) -> Finance
Deutsche Bahn AG -> Transportation and Logistics
NCR -> Software and IT Services
Beiersdorf AG -> Consumer Goods
CBRE Group -> Real Estate
BMW Group -> 

In [152]:
df_comparison = pd.DataFrame()
df_comparison["Company"] = random_companies
df_comparison["Predicted Industry"] = industries
df_comparison["Actual Industry"] = actual_industries

# Compare the predicted industries with the actual industries
df_comparison["Correct"] = df_comparison["Predicted Industry"] == df_comparison["Actual Industry"]

# Count the number of correct predictions
correct_predictions = df_comparison["Correct"].sum()
print(f"Accuracy: {correct_predictions/len(companies)}")
df_comparison


Accuracy: 0.7039106145251397


,Company,Predicted Industry,Actual Industry,Correct
0,Fresenius SE & Co. KGaA,Healthcare,Healthcare,True
1,Nationwide,Insurance,Finance,False
2,Deere,Manufacturing,Manufacturing,True
3,Dollar General,Retail,Consumer Goods,False
4,BASF SE,Chemicals,Manufacturing,False
...,...,...,...,...
195,ICICI Bank,Finance,Finance,True
196,Hindustan Zinc,Energy and Mining,Energy and Mining,True
197,RWE AG,Energy and Mining,Energy and Mining,True
198,Baker Hughes,Energy and Mining,Energy and Mining,True


# Results 

The folowing are the results:

1. Entries: 50, Accuracy: 60%, Time: 1:37 minutes
2. Entries: 100, Accuracy: 59%, Time: 2:22 minutes
3. Entriess 200, Accuracy 59%, Time: 5:03 minutes

When testing with the random company approach 

1. Entries 200, Accuracy 65.8%, Time: 5:47 minutes
2. Entries 200, Accuracy 61.1%, Time: 4:50 minutes
3. Entries 200, Accuracy 70.4%, Time: $:50 minutes

# Create a dictionary of predicted industries

Using the correctly predicted industries we will create a dictionary mapping companies to correctly predicted industries

In [ ]:
company_dict = {}

{'Phillips 66': 'Energy and Mining', 'Meta Platforms': 'Software and IT Services', 'Occidental Petroleum': 'Energy and Mining', 'Vertex Pharmaceuticals': 'Healthcare', 'Reinsurance Group of America': 'Finance', 'Axis Bank': 'Finance', 'Grafton Group': 'Construction', 'Genworth Financial': 'Finance', 'Scottish Mortgage Investment Trust': 'Finance', 'BMW Group': 'Transportation and Logistics', 'First American Financial': 'Finance', 'Aditya Birla Capital': 'Finance', 'Kotak Mahindra Bank': 'Finance', 'Reynolds American Inc.': 'Consumer Goods', 'Deutsche Pfandbriefbank AG': 'Finance', 'FirstEnergy': 'Energy and Mining', 'Devon Energy': 'Energy and Mining', 'Hikma Pharmaceuticals': 'Healthcare', 'Northwestern Mutual': 'Finance', 'E.ON SE': 'Energy and Mining', 'SVB Financial Group': 'Finance', 'Indian Oil Corporation': 'Energy and Mining', 'Lululemon athletica': 'Consumer Goods', 'Massachusetts Mutual Life Insurance': 'Finance', 'Daimler AG (now known as Mercedes-Benz AG)': 'Transportation 

In [153]:
for index, row in df_comparison.iterrows(): # Iterate over the rows
    if row["Correct"] == True: # If the prediction was correct
        if row["Company"] not in company_dict: # If the company is not already in the dictionary
            company_dict[row["Company"]] = row["Predicted Industry"] # Add the company and the predicted industry to the dictionary


In [154]:
print(company_dict)

{'Phillips 66': 'Energy and Mining', 'Meta Platforms': 'Software and IT Services', 'Occidental Petroleum': 'Energy and Mining', 'Vertex Pharmaceuticals': 'Healthcare', 'Reinsurance Group of America': 'Finance', 'Axis Bank': 'Finance', 'Grafton Group': 'Construction', 'Genworth Financial': 'Finance', 'Scottish Mortgage Investment Trust': 'Finance', 'BMW Group': 'Transportation and Logistics', 'First American Financial': 'Finance', 'Aditya Birla Capital': 'Finance', 'Kotak Mahindra Bank': 'Finance', 'Reynolds American Inc.': 'Consumer Goods', 'Deutsche Pfandbriefbank AG': 'Finance', 'FirstEnergy': 'Energy and Mining', 'Devon Energy': 'Energy and Mining', 'Hikma Pharmaceuticals': 'Healthcare', 'Northwestern Mutual': 'Finance', 'E.ON SE': 'Energy and Mining', 'SVB Financial Group': 'Finance', 'Indian Oil Corporation': 'Energy and Mining', 'Lululemon athletica': 'Consumer Goods', 'Massachusetts Mutual Life Insurance': 'Finance', 'Daimler AG (now known as Mercedes-Benz AG)': 'Transportation 

In [155]:
print(len(company_dict))

257


In [156]:
# Save to a JSON file 
import json
with open("prediction_dictioanry.json", "w") as f:
    json.dump(company_dict, f, indent=4)